In [1]:
import pandas as pd
import numpy as np

from surprise import Dataset, Reader, SVD, NormalPredictor
from surprise.model_selection import cross_validate, GridSearchCV, PredefinedKFold
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import train_test_split as sur_tts

from sklearn.model_selection import train_test_split

In [49]:
# import ratings csv as df

ratings = pd.read_csv('../../data/ratings.csv')

#drop unnecessary timestamp

ratings.drop(['timestamp'],inplace=True,axis=1)

ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [50]:
#drop movies with low number of ratings
print(len(ratings.index))
unique_movies = ratings['movieId'].unique()

movie_count = {}

for i in unique_movies:
    movie_count[i]= 0
    
for i in ratings['movieId']:
    movie_count[i] += 1

rare_movies = []
for movie in movie_count.keys():
    if movie_count[movie] <= 15:
        rare_movies.append(movie)
        
to_delete = []
for index, row in ratings.iterrows():
    if row.movieId in rare_movies:
        to_delete.append(index)
        
ratings.drop(to_delete,inplace=True)
print(len(ratings.index))

100836
72675


In [51]:
#use sklearn library train test split
y = pd.DataFrame(ratings['rating'])
X= ratings.drop('rating',axis=1)

X_train,X_test,y_train,y_test = train_test_split(X,y)

#remerge to convert using reader

train_df = pd.concat([X_train,y_train],axis=1)

test_df = pd.concat([X_test,y_test],axis=1)

#convert to surprise dataframes
reader = Reader(line_format='user item rating', sep=',')

trainset= Dataset.load_from_df(train_df, reader=reader)

testset = Dataset.load_from_df(test_df, reader=reader)

#make testset data usable for testing

blank,testset = sur_tts(testset, test_size=.95)

type(testset)

list

In [39]:
param_grid = {'n_factors':[250,350],'n_epochs':[30],
              'lr_all':[.05],'reg_all':[.06]}

gs_svd = GridSearchCV(SVD,param_grid,measures=['rmse'],cv=3,n_jobs=-1)
gs_svd.fit(trainset)

params = gs_svd.best_params['rmse']

params

{'n_factors': 350, 'n_epochs': 30, 'lr_all': 0.05, 'reg_all': 0.06}

In [40]:
params = gs_svd.best_params['rmse']

params

{'n_factors': 350, 'n_epochs': 30, 'lr_all': 0.05, 'reg_all': 0.06}

In [52]:


algo = SVD(n_factors= 350, n_epochs=30, lr_all= 0.05, reg_all= 0.06)
train_set = trainset.build_full_trainset()
algo.fit(train_set)

preds = algo.test(testset)

In [53]:
accuracy.rmse(preds)

RMSE: 0.8266


0.826644822396344